## Oddity File Transfer (.ps1)

VERSION: `Oddity_Auto_Sync_General`

### Advanced Explanation

This PowerShell script is designed for selective directory transfer to a remote server. It commences by capturing the current date and prompting the user for their username. The script then constructs a remote path (`/home/$username/scratch/Final_Oddity/`) and asks the user to confirm or adjust this path.

Directories created in the past 24 hours are identified using `Get-ChildItem -Directory` combined with a `Where-Object` filter based on creation time. For each directory found, the script calculates its size in kilobytes and prompts the user to confirm its transfer to the server. The transfer is executed using `scp` (Secure Copy Protocol) to the specified remote path. The script monitors the success of each transfer, keeping a count of successful transfers and listing the names of transferred directories. It concludes with a summary of the transfer process and awaits the user's input to close.

### User-Friendly Explanation

This script helps you back up newly created folders from your computer to a server. First, it asks for your username and verifies the server path for the backup. Then, it looks for any folders created in the last day and tells you their sizes, asking if you want to transfer each one to the server.

If you agree, the script securely sends the folder to the specified server path. It keeps track of how many folders it transfers and lists their names once done. At the end, the script informs you that the transfer is complete and waits for you to press Enter before closing. This tool is handy for regularly backing up your most recent folders to a secure location.

### Oversimplified Explanation

This should be kept and run where pre-processed Oddity session folders are backed up. It will send new ones from the local device to your scratch folder on Compute Canada. 


### Note
    
Track meta data before sending and compare it to the output of the check scripts. 

In [ ]:
$currentDate = Get-Date

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/Final_Oddity/"
Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name
    $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1KB

    $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($directorySize) KB] to the server at `"$remotePath`"? [Y/N]"

    if ($userInput -eq "Y") {

        scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}

Read-Host -Prompt "Press Enter to Close"


VERSION: `Oddity_Auto_Sync_(name).ps1`

Replace remote pathbase and scp command to reflect the user and save with your name using Powershell ISE. Note that there will be no prompts as the script is designed for maximum efficiency so make sure you know what you are doing. 

In [ ]:
$currentDate = Get-Date

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/Actual_Final_Oddity/"

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
    if ($?) {
        $transferredCount++
        $transferredDirectories += $dir.Name
        
        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
    } else {
        Write-Warning "Failed to transfer directory: $dir"
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}
